In [1]:
!pip install torch torchvision

In [1]:
#Transformer architecture to do NLP task
!pip install transformers torch
!pip install diffusers
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.9 MB/s eta 0:00:00


In [7]:
%%writefile Hugging_face.py
# Importing necessary packages
import torch
from transformers import BartForConditionalGeneration, BartTokenizer
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from diffusers import StableDiffusionPipeline
import streamlit as st
import re
import matplotlib.pyplot as plt


st.title(':black["Hugging Face Model Hub: Real-Time NLP in Action"]')
st.write('''Explore NLP tasks like sentiment analysis, text generation, and more with Hugging Face models in real-time.''')
# Task selection
task = st.sidebar.selectbox('Choose a task', ['Text Summarization', 'Next Word Prediction',
                                              'Story Prediction', 'Chatbot',
                                              'Sentiment Analysis', 'Question Answering',
                                              'Image Generation'])

if task=='Text Summarization':
  user_input = st.text_area(f'Enter input for {task}')
  if st.button('Submit'):
    with st.spinner(f"Processing {task}..."):
      model_name = "facebook/bart-large-cnn"
      tokenizer = BartTokenizer.from_pretrained(model_name)
      model = BartForConditionalGeneration.from_pretrained(model_name)

      # Set the model to evaluation mode
      model.eval()

      def summarize(text, model, tokenizer, max_length=200, min_length=30, do_sample=False):
        inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)
        summary_ids = model.generate(inputs, max_length=max_length, min_length=min_length, length_penalty=2.0, num_beams=4, early_stopping=True)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return summary

      summary = summarize(user_input, model, tokenizer)
      st.write("\nSummary:\n", summary)

elif task=='Next Word Prediction':
  user_input = st.text_area(f'Enter input for {task}')
  if st.button('Submit'):
    # Load the tokenizer and model
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    model = GPT2LMHeadModel.from_pretrained('gpt2')

    # Set the model to evaluation mode
    model.eval()
    def predict_next_word(prompt, model, tokenizer, top_k=5):
      # Tokenize input text
      inputs = tokenizer(prompt, return_tensors='pt')

      # Generate predictions
      with torch.no_grad():
        outputs = model(**inputs)

      # Get logits of the last token in the sequence
      next_token_logits = outputs.logits[:, -1, :]

      # Filter the top k tokens by their probability
      top_k_tokens = torch.topk(next_token_logits, top_k).indices[0].tolist()

      # Decode the top k tokens to their corresponding words
      predicted_tokens = [tokenizer.decode([token]) for token in top_k_tokens]

      return predicted_tokens
    predicted_words = predict_next_word(user_input, model, tokenizer)
    st.write(f"Next word predictions: {predicted_words}")

elif task=='Story Prediction':
  user_input = st.text_area(f'Enter input for {task}')
  if st.button('Submit'):
    with st.spinner(f"Processing {task}..."):
      story_predictor = pipeline('text-generation', model='gpt2')
      prediction = story_predictor(user_input,max_length=1000,truncation=True,clean_up_tokenization_spaces=True)
      st.write(prediction[0]['generated_text'])

elif task=='Chatbot':
  # Load pre-trained model and tokenizer with padding set to left
  model_name = "microsoft/DialoGPT-medium"
  tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left')
  model = AutoModelForCausalLM.from_pretrained(model_name)

  # Set the model to evaluation mode
  model.eval()

  def chat_with_model(prompt, chat_history_ids, model, tokenizer):
    # Encode the new user input, add the eos_token and return a tensor in PyTorch
    new_user_input_ids = tokenizer.encode(prompt + tokenizer.eos_token, return_tensors='pt')

    # Append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if chat_history_ids is not None else new_user_input_ids

    # Generate a response
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # Decode the response and print it
    response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

    return response, chat_history_ids

  # Initialize chat history
  chat_history_ids = None

  st.write("Start chatting with the bot (type 'quit' to stop)")
  keyz = 's1'
  user_input = st.text_input("You: ",key = keyz)
  while user_input.lower() != 'quit':
    response, chat_history_ids = chat_with_model(user_input, chat_history_ids, model, tokenizer)
    if (response != '')and(user_input != ''):
      st.write(f"Bot: {response}")
      keyz += '1'
      user_input = st.text_input("You: ",key = keyz)
    #response = ''

elif task=='Sentiment Analysis':
  user_input = st.text_area(f'Enter input for {task}')
  if st.button('Submit'):
    with st.spinner(f"Processing {task}..."):
      # Load pre-trained sentiment-analysis pipeline
      sentiment_analysis = pipeline("sentiment-analysis")

      def analyze_sentiment(texts):
        results = sentiment_analysis(texts)
        for text, result in zip(texts, results):
          st.write(f"Text: {text}")
          st.write(f"Sentiment: {result['label']}, Score: {result['score']:.4f}")

      # Example texts for sentiment analysis
      sent_texts = re.split(r'([.!?])', user_input)
      resultz = [sent_texts[i] + sent_texts[i+1] for i in range(0, len(sent_texts)-1, 2)]
      # Perform sentiment analysis
      for text in resultz:
        analyze_sentiment(text)

elif task=='Question Answering':
  user_input = st.text_area(f'Enter input for {task}')
  question = st.text_input('Enter a Question ')
  if st.button('Submit'):
    with st.spinner(f"Processing {task}..."):
      # Load the pre-trained model and tokenizer
      model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
      tokenizer = AutoTokenizer.from_pretrained(model_name)
      model = AutoModelForQuestionAnswering.from_pretrained(model_name)

      def answer_question(question, context):
        # Tokenize input question and context
        inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")

        # Get input ids and attention mask
        input_ids = inputs["input_ids"].tolist()[0]

        # Get the model's output
        outputs = model(**inputs)
        answer_start_scores = outputs.start_logits
        answer_end_scores = outputs.end_logits

        # Get the most likely beginning and end of answer with the argmax of the score
        answer_start = torch.argmax(answer_start_scores)
        answer_end = torch.argmax(answer_end_scores) + 1

        # Convert tokens to the answer
        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

        return answer

      # Define a context and a question
      context = user_input

      # Get the answer
      answer = answer_question(question, context)
      st.write(f"Answer: {answer}")

elif task=='Image Generation':
  user_input = st.text_area(f'Enter input for {task}')
  if st.button('Submit'):
    with st.spinner(f"Processing {task}..."):
      # Load the Stable Diffusion model
      pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4").to("cuda")

      # Function to generate images
      def generate_image(prompt):
        image = pipe(prompt).images[0]
        return image
      # Example text prompt
      prompt = user_input

      # Generate image
      image = generate_image(prompt)

      # Display the generated image
      st.image(image)




Overwriting Hugging_face.py


In [8]:
!npm install localtunnel


up to date, audited 23 packages in 2s

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [9]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))


Password/Enpoint IP for localtunnel is: 34.125.216.181


In [10]:
!streamlit run /content/Hugging_face.py &>/content/logs.txt &


In [ ]:

!npx localtunnel --port 8501

your url is: https://ripe-brooms-wait.loca.lt
